In [3]:
import pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import *
import numpy as np
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.ml.feature import *
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
import copy
import math
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import pandas as pd

entry2 = None
entry3 = None
entry4 = None

def standardize(dfc):
    assembler = VectorAssembler(inputCols=list(set(dfc.columns) - set(["player_id", "preferred_foot", "work_rate", "position", "sub_position", "age_copy"])), 
                                outputCol="features")

    dfc = assembler.transform(dfc)

    scaler = StandardScaler(inputCol="features", 
                            outputCol="scaled_features",
                            withStd=True, withMean=True)

    # Compute summary statistics by fitting the StandardScaler
    scalerModel = scaler.fit(dfc)

    # Normalize each feature to have unit standard deviation
    dfc = scalerModel.transform(dfc)
    return dfc

def apply_one_hot_encoding(df, column_names, df_id):

    def generate_encoded_columns(df, name_column, column_values):
        return [
            when(df[name_column].isin(value), 1).otherwise(0).alias(f"{name_column}_{value}")
            for value in column_values
        ]
    diz = {}
    for col in column_names:
        diz[col]=[row[col] for row in df.select(col).distinct().collect()]
    encoded_columns = []
    for name_column, column_values in diz.items():
        encoded_columns.extend(generate_encoded_columns(df, name_column, column_values))
    encoded_df = df.select(df_id, *encoded_columns)
    #join the encoded df with the original df on the key df_id
    df = df.join(encoded_df, df_id)
    #drop the columns in column_names
    df = df.drop(*column_names)
    return df

def dataset_assembler(df): 
   # create an instance of VectorAssembler specifying the input columns and the output column
    assembler = VectorAssembler(inputCols=list(set(df.columns) - set(["player_id", "features"])), outputCol="pcaFeatures")

    # apply the VectorAssembler to the DataFrame
    df_pca = assembler.transform(df)
    return df_pca

def get_optimal_k(df, desired_variance_percentage, k):
    
    assembler = VectorAssembler(inputCols=list(set(df.columns) - set(["player_id", "features", "age_copy"])), outputCol="features_")
    df_assembled = assembler.transform(df)

    pca = PCA(k=k, inputCol="features_", outputCol="pcaFeatures")
    model = pca.fit(df_assembled)

    explained_variance = model.explainedVariance.toArray()
    total_variance = np.sum(explained_variance)

    cumulative_variance = 0.0
    num_selected_features = 0

    for variance in explained_variance:
        cumulative_variance += variance
        num_selected_features += 1
        if cumulative_variance >= desired_variance_percentage * total_variance:
            break

    return num_selected_features


def apply_PCA(df, features_name_list, k):

    assembler = VectorAssembler(inputCols=features_name_list, outputCol="features_")
    df_assembled = assembler.transform(df)
    pca = PCA(k=k, inputCol="features_", outputCol="pcaFeatures")
    model = pca.fit(df_assembled)
    df_pca = model.transform(df_assembled).select("player_id", "pcaFeatures")
    
    return df_pca


def flag_PCA(df,flag,dfc): # if flag is True, apply PCA, otherwise apply assembler
    if flag is True:
        desired_variance_percentage = 0.95
        k = len(list(set(df.columns) - set(["player_id", "features", "age_copy"]))) + len(dfc.first()["scaled_features"])
        optimal_k = get_optimal_k(df, desired_variance_percentage, k-1)
        #print("I'm applying pca with an optimal k= ", optimal_k)
        df_pca = apply_PCA(df, list(set(df.columns)-{"player_id", "features", "age_copy"}), optimal_k)
    else:
        df_pca = dataset_assembler(df)
        print("I'm not applying pca, but i'm assembling the dataset for k-means")
    return df_pca

def k_means(dataset, n_clusters, distance_measure="euclidean", max_iter=150, features_col="pcaFeatures", prediction_col="cluster", random_seed=42):
  
    #print("""k-means parameters: - number of clusters = {:d} - max iterations = {:d} - distance measure = {:s} - random seed = {:d}""".format(n_clusters, max_iter, distance_measure, random_seed))

    # Train a K-means model
    kmeans = KMeans(featuresCol=features_col, 
                    predictionCol=prediction_col, 
                    k=n_clusters, 
                    initMode="k-means||", # or random
                    initSteps=20, 
                    tol=0.00000001, 
                    maxIter=max_iter, 
                    seed=random_seed, 
                    distanceMeasure=distance_measure)
    model = kmeans.fit(dataset)

    # Make clusters
    clusters_df = model.transform(dataset)

    return model, clusters_df

def applying_elbow(df, features_col="pcaFeatures", prediction_col="cluster", min_k=60, max_k=100, random_seed=42):        

    clustering_results = {}
    k_values = range(min_k, max_k)

    silhouette_values = []

    for k in k_values:
        # create the k-means model
        model, clusters_df = k_means(df, k, features_col="pcaFeatures", prediction_col=prediction_col, random_seed=42)
        clusters_df = clusters_df.withColumn("features", clusters_df["pcaFeatures"])

        # Calcolo del Silouette Score
        silhouette = evaluate_k_means(clusters_df)
        silhouette_values.append(silhouette)
        clustering_results[k] = silhouette

    return clustering_results

def evaluate_k_means(clusters, metric_name="silhouette", distance_measure="squaredEuclidean",prediction_col="cluster"):
  
  # Evaluate clustering by computing Silhouette score
  evaluator = ClusteringEvaluator(metricName=metric_name,
                                  distanceMeasure=distance_measure, 
                                  predictionCol=prediction_col
                                  )

  return evaluator.evaluate(clusters)


# split dfc in two dataframes: one composed of player with age >26, the other composed of player with age <=26
def split_dataframe(dfc):
    dfc1 = dfc.filter(dfc.age_copy > 25)
    dfc2 = dfc.filter(dfc.age_copy <= 25)
    return dfc1, dfc2

# def funtion to remove columns in df_k_means1 and df_k_means2
def remove_columns(df_k_means):
    # drop pcaFeatures, features
    df_k_means = df_k_means.drop("pcaFeatures")
    df_k_means = df_k_means.drop("features")
    df_k_means = df_k_means.drop("nationality_name")
    df_k_means = df_k_means.drop("work_rate")
    df_k_means = df_k_means.drop("weight_kg")
    return df_k_means


def search_cluster(df1, df2, player_id):
    # convert pysspark dataframe to pandas dataframe
    pdf1 = df1.filter(col("player_id") == player_id).toPandas()
    pdf2 = df2.toPandas()

    # extract the data for player_id from pdf1
    player_id_dict = pdf1.drop(columns=["player_id", "name", "age", "cluster"]).iloc[0].to_dict()

    # find unique clusters in pdf2
    unique_clusters = pdf2["cluster"].unique()
    clusters_value = []

    for cluster in unique_clusters:
        # filter rows where column cluster is equal to the current cluster value
        pdf_filtered = pdf2[pdf2["cluster"] == cluster]
        pdf_filtered = pdf_filtered.drop(columns=["player_id", "name", "age", "cluster"])

        # compute mean values for numeric columns and mode values for non numeric columns
        clus_values = pdf_filtered.mean(numeric_only=True).to_dict()
        non_numeric_columns = pdf_filtered.select_dtypes(exclude=["number"]).columns

        for column in non_numeric_columns:
            clus_values[column] = pdf_filtered[column].mode()[0]

        clusters_value.append(clus_values)

    return player_id_dict, clusters_value


def find_most_similar_index(d, dlist):
    categorical_keys = [key for key, value in d.items() if isinstance(value, str)]
    numeric_keys = [key for key in d if key not in categorical_keys]

    # convert categorical variables into one-hot encoding
    encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
    categorical_values = encoder.fit_transform([[d[key]] for key in categorical_keys])

    # build vectors including numeric attributes and one-hot encoding
    vectors = []
    for d_dict in dlist:
        numeric_values = [d_dict.get(key, 0.0) for key in numeric_keys]
        cat_values = categorical_values.flatten().tolist()
        vector = numeric_values + cat_values
        vectors.append(vector)

    vector = [d.get(key, 0.0) for key in numeric_keys] + categorical_values.flatten().tolist()

    # compute the cosine similarity between vectors
    similarities = cosine_similarity(vectors, [vector])
    most_similar_index = np.argmax(similarities)
    return most_similar_index

def get_clus(player_dict, clusters_list, min_value, max_value): # return the list of the clusters for the player of player_dict
    player_dict_c = copy.deepcopy(player_dict)
    clusters_list_c = copy.deepcopy(clusters_list)

    clus_list_first = [] # dictionary that have passed the first filter

    # first filter
    for i in range(len(clusters_list_c)):
        if clusters_list_c[i]["position"] == player_dict_c["position"] and clusters_list_c[i]["sub_position"] == player_dict_c["sub_position"] and clusters_list_c[i]["preferred_foot"] == player_dict_c["preferred_foot"]:
            height_diff = int(clusters_list_c[i]["height_cm"]) - int(player_dict_c["height_cm"])
            skills_diff = int(clusters_list_c[i]["skill_moves"]) - int(player_dict_c["skill_moves"])
            if math.sqrt(height_diff**2) < 20 and math.sqrt(skills_diff**2) <= 2 and int(clusters_list_c[i]["last_valuation"]) <= max_value and int(clusters_list_c[i]["last_valuation"]) >= min_value:
                clus_list_first.append((i,clusters_list_c[i]))

    # delete from clus_list_first and player_dict_c keys height_cm, skill_moves, last_valuation, position, sub_position, preferred_foot

    keys_todelete = ["height_cm", "skill_moves", "last_valuation", "position", "sub_position", "preferred_foot"]

    for clus in clus_list_first:
        for key in keys_todelete:
            del clus[1][key]
    #print(clus_list_first)

    # delete from player_dict_c keys height_cm, skill_moves, last_valuation, position, sub_position, preferred_foot
    for key in keys_todelete:
        del player_dict_c[key]

    # sort the player_dict_c keys based on the values
    sorted_keys_player_id = sorted(player_dict_c, key=player_dict_c.get, reverse=True)
    #print(sorted_keys_player_id)

    # sort the clusters based on the values
    sorted_keys_clusters = []
    for clus in clus_list_first:
        sorted_keys_clusters.append((clus[0], sorted(clus[1], key=clus[1].get, reverse=True)))
    #print(sorted_keys_clusters)

    # filter the filtered clusters based on the remaining stats value
    list_to_return = []
    for clus in sorted_keys_clusters:
        if len(list((set(sorted_keys_player_id[:10])) & (set(clus[1][:10])))) > 4:
            list_to_return.append(clus[0])

    # the first element of the output list must be the cluster with the smallest market_value
    if len(list_to_return) > 0:
        list_to_return = sorted(list_to_return, key=lambda x: clusters_list[x]["last_valuation"], reverse=False)

    return list_to_return

def from_df_to_dict(df):
    # Convert the DataFrame into a list of rows using collect()
    rows = df.collect()

    # Define the function for the row processing
    def process_row(row):
        column_names = row.__fields__
        row_dict = {column: getattr(row, column) for column in column_names}
        l_diz_result.append(row_dict)

    # Initialize the list for the results
    l_diz_result = []

    # Function application to each row
    for row in rows:
        process_row(row)
    return l_diz_result

def sort_dicts(diz, ldiz):
    del diz["player_id"]
    del diz["cluster"]

    # Creation of the list of unique keys
    keys = set(diz.keys())
    for ld in ldiz:
        keys.update(ld.keys())

    for el in ldiz:
        del el["player_id"]
        del el["cluster"]
    
    # Creation of the extended one-hot matrices for the reference dictionary (diz) and the dictionaries in the list (ldiz)
    diz_matrix = create_extended_one_hot_matrix(diz, keys)
    ldiz_matrices = [create_extended_one_hot_matrix(ld, keys) for ld in ldiz]
    
    # Calculate the cosine similarities between the reference dictionary and the dictionaries in the list
    similarities = [cosine_similarity(diz_matrix, ld_matrix)[0][0] for ld_matrix in ldiz_matrices]

    # Sort the dictionaries in the list based on the similarities
    sorted_ldiz = [ld for _, ld in sorted(zip(similarities, ldiz), reverse=True)]
    
    return sorted_ldiz

def create_extended_one_hot_matrix(diz, keys):
    matrix = np.zeros((1, len(keys)))
    key_list = list(keys)
    for i, key in enumerate(key_list):
        if key in diz:
            value = diz[key]
            if isinstance(value, str):
                matrix[0][i] = 1
            elif isinstance(value, (int, float)):
                matrix[0][i] = value
            elif isinstance(value, (list, tuple)):
                matrix[0][i] = len(value)
    return matrix

def run(df, name_player, min_value, max_value):

    existing_spark_session = SparkSession.getActiveSession()
    if existing_spark_session is not None:
        existing_spark_session.stop()

    # Create the session
    conf = SparkConf(). \
        set('spark.ui.port', "4050"). \
        set('spark.executor.memory', '15G'). \
        set('spark.driver.memory', '50G'). \
        set('spark.driver.maxResultSize', '40G'). \
        setAppName("PySparkProject"). \
        set('spark.executor.cores', "10"). \
        setMaster("local[*]")

    sc = pyspark.SparkContext.getOrCreate(conf=conf)
    spark = SparkSession.builder.getOrCreate()

    sc._conf.getAll()


    df = spark.read.csv(df, header=True, inferSchema=True)

    #obtain from df the "player_id" of the instances with name name_player
    player_id = df.filter(df.name == name_player).select("player_id").collect()[0][0]

    # convert from double to int the features goalkeeping_speed, pace, shooting, passing, dribbling, defending, physic
    df = df.withColumn("goalkeeping_speed", df.goalkeeping_speed.cast("int"))
    df = df.withColumn("pace", df.pace.cast("int"))
    df = df.withColumn("shooting", df.shooting.cast("int"))
    df = df.withColumn("passing", df.passing.cast("int"))
    df = df.withColumn("dribbling", df.dribbling.cast("int"))
    df = df.withColumn("defending", df.defending.cast("int"))
    df = df.withColumn("physic", df.physic.cast("int"))
    df = df.withColumn("mentality_composure", df.mentality_composure.cast("int"))

    # drop column player_id
    df = df.drop("games_played_club", "games_won_club", "games_draw_club", "games_lost_club")

    # drop the features nationality_name and name
    dfc = df.drop("nationality_name", "name")

    # crea una copia della feature age in age_copy
    dfc = dfc.withColumn("age_copy", dfc.age)

    dfc = standardize(dfc)

    dfc = dfc.drop(*list(set(df.columns) - set(["player_id", "preferred_foot", "work_rate", "position", "sub_position", "age_copy"])))

    column_names = ["preferred_foot", "work_rate", "position", "sub_position"] #columns to apply one hot encoding (categorical features)
    dfc = apply_one_hot_encoding(dfc, column_names, "player_id") # apply one hot encoding to the specified columns on the df id player_id

    dfc1, dfc2 = split_dataframe(dfc)

    df1_pca = flag_PCA(dfc1,True,dfc)
    df2_pca = flag_PCA(dfc2,True,dfc)

    # def applying_elbow(df, features_col="pcaFeatures", prediction_col="cluster", min_k=60, max_k=100, random_seed=RANDOM_SEED):
    clustering_results1 = applying_elbow(df1_pca, features_col="pcaFeatures", prediction_col="cluster", min_k=60, max_k=65, random_seed=42)
    clustering_results2 = applying_elbow(df2_pca, features_col="pcaFeatures", prediction_col="cluster", min_k=68, max_k=74, random_seed=42)

    # obtain the key of the max items of clustering_results
    k_clustering1 = list(dict(sorted(clustering_results1.items(), key=lambda item: item[1], reverse=True)))[0]
    k_clustering2 = list(dict(sorted(clustering_results2.items(), key=lambda item: item[1], reverse=True)))[0]

    model1, df_clusters1 = k_means(df1_pca, k_clustering1, features_col="pcaFeatures", prediction_col="cluster", random_seed=42) # k_clustering or we can choose the number of clusters
    model2, df_clusters2 = k_means(df2_pca, k_clustering2, features_col="pcaFeatures", prediction_col="cluster", random_seed=42) # k_clustering or we can choose the number of clusters

    df_clusters1 = df_clusters1.withColumn("features", df_clusters1["pcaFeatures"]) # we need to do this because the function evaluate_k_means needs a column named "features"
    df_clusters2 = df_clusters2.withColumn("features", df_clusters2["pcaFeatures"]) # we need to do this because the function evaluate_k_means needs a column named "features"
    silhouette1 = evaluate_k_means(df_clusters1, metric_name="silhouette", distance_measure="squaredEuclidean", prediction_col="cluster")
    silhouette2 = evaluate_k_means(df_clusters2, metric_name="silhouette", distance_measure="squaredEuclidean", prediction_col="cluster")

    #join the clusters_df with the original df on the key player_id
    df_k_means1 = df.join(df_clusters1, "player_id")
    df_k_means2 = df.join(df_clusters2, "player_id")

    #oridinare df_k_means per cluster
    df_k_means1 = df_k_means1.orderBy("cluster")
    df_k_means2 = df_k_means2.orderBy("cluster")

    df_k_means1 = remove_columns(df_k_means1)
    df_k_means2 = remove_columns(df_k_means2)

###################### SCOUTING ######################

    player_id_dict, clusters_value = search_cluster(df_k_means1, df_k_means2, player_id)
    result_playerid = df_k_means1.filter(df_k_means1.player_id == player_id)

###################### SCOUTING 1 ######################

    player_id_dict_copy = copy.deepcopy(player_id_dict)
    clusters_value_copy = copy.deepcopy(clusters_value)
    keys_to_remove = ["overall", "weak_foot", "pace", "shooting", "dribbling", "defending", "physic", "last_valuation"]

    for key in keys_to_remove:
        del player_id_dict_copy[key]

    for i in range(len(clusters_value_copy)):
        for key in keys_to_remove:
            del clusters_value_copy[i][key]

    if df_k_means2.filter(col("cluster") == 0).count() > 0:
        cluster_index = find_most_similar_index(player_id_dict_copy, clusters_value_copy)
    else:
        cluster_index = find_most_similar_index(player_id_dict_copy, clusters_value_copy) + 1

    print("output cluster index first algorithm:")
    print(cluster_index)

    result1 = df_k_means2.filter(col("cluster") == int(cluster_index))

###################### SCOUTING 2 ######################

    key_list = []
    for k,v in player_id_dict.items():
        key_list.append(k)

    # remove: overall, height_cm, skill_moves, pace, shooting, dribbling, defending, physic from key_list
    key_list = list(set(key_list) - set(["overall", "weak_foot", "pace", "shooting", "dribbling", "defending", "physic"]))

    player_id_dict_c = player_id_dict.copy()
    clusters_value_c = copy.deepcopy(clusters_value)
    keys_to_remove = ["overall", "weak_foot", "pace", "shooting", "dribbling", "defending", "physic"]

    for key in keys_to_remove:
        del player_id_dict_c[key]

    for i in range(len(clusters_value_c)):
        for key in keys_to_remove:
            del clusters_value_c[i][key]

    l = get_clus(player_id_dict_c, clusters_value_c, min_value, max_value)

    if (len(l) == 0):
        pass
    else:
        print("output cluster index second algorithm:")
        print(l[0])
        result2 = df_k_means2.filter(col("cluster") == l[0])

    #####################################################################
    column_names = result_playerid.columns
    diz_result_playerid = {column: value for column, value in zip(column_names, result_playerid.first())}

    l_diz_result1 = from_df_to_dict(result1)
    if len(l) != 0:
        l_diz_result2 = from_df_to_dict(result2)

    if len(l) != 0:
        l_diz_result2 = sort_dicts(copy.deepcopy(diz_result_playerid), copy.deepcopy(l_diz_result2))
    l_diz_result1 = sort_dicts(copy.deepcopy(diz_result_playerid), copy.deepcopy(l_diz_result1))


    print("OUTPUT PLAYER FIRST ALGORITHM (PLAYERS ARE SORTED FROM MOST SIMILAR TO LEAST SIMILAR):")
    result1 = spark.createDataFrame(l_diz_result1)
    print(result1.select("name",*result1.columns).show())

    if len(l) != 0:
        print("OUTPUT PLAYER SECOND ALGORITHM (PLAYERS ARE SORTED FROM MOST SIMILAR TO LEAST SIMILAR):")
        result2 = spark.createDataFrame(l_diz_result2)
        print(result2.select("name",*result2.columns).show())
    else :
        print("No players found with the second algorithm")



def run_button_click():
    # obtain the values inserted by the user
    param2 = str(entry2.get())
    param3 = int(entry3.get())
    param4 = int(entry4.get())

    # run the function run with the inserted parameters
    run("task2dataset.csv", param2, param3, param4)
    messagebox.showinfo("Done!", "Execution completed!")


def main():
    # Gui creation
    root = tk.Tk()
    root.title("Insert parameters")
    root.geometry("700x350")

    global entry2, entry3, entry4

    entry_font = ("Arial", 12) 

    # labels
    label2 = tk.Label(root, text="Player Name:", font=("Arial", 14))
    label2.pack()
    label3 = tk.Label(root, text="Min Price:", font=("Arial", 14))
    label3.pack()
    label4 = tk.Label(root, text="Max Price:", font=("Arial", 14))
    label4.pack()

    # text cell for inserting the parameters
    entry2 = tk.Entry(root, width=30, font=entry_font)
    entry2.pack()
    entry3 = tk.Entry(root, width=30, font=entry_font)
    entry3.pack()
    entry4 = tk.Entry(root, width=30, font=entry_font)
    entry4.pack()

    # button to run the function run
    button_font = ("Arial", 12)
    run_button = tk.Button(root, text="Search", command=run_button_click, width=20, height=2, font=button_font)
    run_button.pack()

    # GUI start
    root.mainloop()

if __name__ == '__main__':
    main()

c:\Users\marco\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


output cluster index first algorithm:
18
output cluster index second algorithm:
14
OUTPUT PLAYER FIRST ALGORITHM (PLAYERS ARE SORTED FROM MOST SIMILAR TO LEAST SIMILAR):
+--------------------+---+------------------+-------------------+--------------------------+-----------------------+-----------------+---------+---------------------------+------------------------+-------------------------+---------+------------------+--------------------+-------------------+-----------------------+--------------------+-----------------+---------+--------------+--------------------+-------------------+-----------------------+-------------------+---------------------+----------------+---------------------+----------------+----------------+------------------+---------------------+--------------------+-------+----+-------+------+--------+-------------+----------------+----------------+-------------+--------------+--------------+--------+------------------+-----------+---------------+-----------------+----